In [1]:
using Pkg
for p in ("Knet","ArgParse","Images","ImageMagick", "MAT")
    haskey(Pkg.installed(),p) || Pkg.add(p)
end
include("resnet.jl")
using Knet, ArgParse, MAT, .ResNet, Images, Random
import Base: length, size, iterate

┌ Warning: `@add_arg_table` is deprecated, use `@add_arg_table!` instead
└ @ ArgParse In[1]:5


In [2]:
model = matconvnet("imagenet-resnet-101-dag")

┌ Info: Loading imagenet-resnet-101-dag.mat...
└ @ Main.ResNet /home/cankucuksozen/.julia/packages/Knet/ALUfa/data/imagenet.jl:14


Dict{String,Any} with 4 entries:
  "meta"   => Dict{String,Any}("classes"=>Dict{String,Any}("name"=>Any["n014407…
  "params" => Dict{String,Any}("name"=>Any["conv1_filter" "bn_conv1_mult" … "fc…
  "vars"   => Dict{String,Any}("name"=>Any["data" "conv1" … "fc1000" "prob"],"p…
  "layers" => Dict{String,Any}("name"=>Any["conv1" "bn_conv1" … "fc1000" "prob"…

In [3]:
struct resnet
    w
    ms
end

In [4]:
function resnet(model)
    w, ms = ResNet.get_params(model["params"], Array{Float32})
    return resnet(w, ms)
end

function (r::resnet)(x)
    res = ResNet.resnet101
    y = res(r.w, x, r.ms)   
    return y
end

function predict(scores)
    pred = [j[1] for j in argmax(scores,dims = 1)]
    return pred
end

function (r::resnet)(x,y)
    scores = r(x)
    loss = nll(scores, y)
    return loss
end
    

In [5]:
function preprocess_img(img, average_img)
    
    new_size = ntuple(i->div(size(img,i)*224,minimum(size(img))),2)
    a1 = Images.imresize(img, new_size)
    i1 = div(size(a1,1)-224,2)
    j1 = div(size(a1,2)-224,2)
    b1 = a1[i1+1:i1+224,j1+1:j1+224]
    c1 = channelview(b1)
    d1 = convert(Array{Float32}, c1)
    e1 = reshape(d1, (224,224,3,1))
    f1 = (255 * e1 .- data.avgimg)
    g1 = permutedims(f1, [2,1,3,4])
    
    return g1

end

preprocess_img (generic function with 1 method)

In [6]:
struct imagenet_minibatch
    classes
    class_to_label
    class_descriptions
    data_path
    samples_list
    batchsize
    shuffle
    mode
    ninstances 
    avgimg
end

In [16]:
function imagenet_minibatch(model; batchsize = 4, shuffle = true, mode = "train")
    
    root_path = "/media/cankucuksozen/CAN HD/Can/image_net/imagenet-object-localization-challenge/ILSVRC/"
    samples_path = joinpath(root_path, "ImageSets/CLS-LOC/")
    data_path = joinpath(root_path, "Data/CLS-LOC/train")
    
    classes = model["meta"]["classes"]["name"]
    labels = reshape(collect(1:length(classes)),(1,:))
    class_to_label = Dict(classes .=> labels)
    
    class_descriptions = model["meta"]["classes"]["description"]
    
    avgimg = model["meta"]["normalization"]["averageImage"]
    avgimg = convert(Array{Float32}, avgimg) 
    
    if mode == "train"
        trn_smp_txt_path = joinpath(samples_path, "train_cls.txt")
        trn_samples_file = open(trn_smp_txt_path)
        trn_samples = readlines(trn_samples_file)
        trn_ninstances = length(trn_samples)
    end
    
    return imagenet_minibatch(classes, class_to_label, class_descriptions, data_path, trn_samples, 
                                batchsize, shuffle, mode, trn_ninstances, avgimg)
end

imagenet_minibatch

In [17]:
function length(d::imagenet_minibatch)
    batch_count, remains = divrem(d.ninstances, d.batchsize)
    return (remains == 0 ? batch_count : batch_count + 1)
end

length (generic function with 194 methods)

In [18]:
function iterate(d::imagenet_minibatch, 
    state=ifelse( d.shuffle, randperm(d.ninstances), collect(1:d.ninstances)))

    inds = state; 
    inds_len = length(inds) 
    max_ind = 0
    
    if inds_len == 0 
        return nothing
    else
        batch_img = []
        batch_label = []
        max_ind = min(inds_len, d.batchsize)
        
        for i in range(1, stop = max_ind)
            samp = d.samples_list[inds[i]]
            class = split(samp, "/")[1]
            label = d.class_to_label[class]
            img_ext = (split(samp, " ")[1]) * ".JPEG"
            img_path = joinpath(d.data_path, img_ext)
            img = Images.load(img_path)
            img = preprocess_img(img, d.avgimg)
            img = KnetArray(img)
            push!(batch_img, img)
            push!(batch_label, label)
        end
        
        img = cat(batch_img..., dims = 4)
        labels = hcat(batch_label...)

        deleteat!(inds, 1:max_ind)

        return ((img,labels), inds)
    end
end

iterate (generic function with 397 methods)

In [19]:
function rand_minibatch(d::imagenet_minibatch)
    
    inds = randperm(d.ninstances)
    batch_img = []
    batch_label = []
    
    for i in range(1, stop = d.batchsize)
        samp = d.samples_list[inds[i]]
        class = split(samp, "/")[1]
        label = d.class_to_label[class]
        img_ext = (split(samp, " ")[1]) * ".JPEG"
        img_path = joinpath(d.data_path, img_ext)
        img = Images.load(img_path)
        img = preprocess_img(img, d.avgimg)
        img = KnetArray(img)
        push!(batch_img, img)
        push!(batch_label, label)
    end
        
    img = cat(batch_img..., dims = 4)
    labels = hcat(batch_label...)

    return (img,labels)
end

rand_minibatch (generic function with 1 method)

In [20]:
resnet101 = resnet(model);
data = imagenet_minibatch(model);

In [21]:
mb = rand_minibatch(data);

In [22]:
x = KnetArray(mb[1])
y = mb[2]

1×4 Array{Int64,2}:
 130  369  342  306

In [23]:
scores = resnet101(x)

1000×4 KnetArray{Float32,2}:
 -0.883756    -2.56079     1.0308    -1.61341 
  1.28838      0.485057    1.17118    2.52391 
 -1.60018      0.507563   -1.13792   -2.17837 
 -2.6587      -3.00223    -0.42736   -3.65635 
 -3.28124     -1.03732    -2.788     -0.827654
 -1.95902     -1.58864    -0.912486   0.690081
 -0.0641573   -0.752815    1.22915    0.862188
  0.537021     0.975576    1.05255    2.90911 
 -2.35892     -1.16602    -1.35082    3.13888 
  0.780459    -0.0830567   1.41386    1.47531 
 -2.78058     -0.287055   -1.57793    2.26129 
  1.25589      0.713824    1.73066    0.440039
  2.06584      0.21744     0.515058   3.93743 
  ⋮                                           
 -1.58394     -1.1853     -1.4788    -0.957123
 -3.81353     -1.28941    -4.14675    0.361152
 -1.73637     -1.73484    -1.64258   -2.21791 
  0.72724     -2.0316      2.05737    1.87932 
 -0.841031     0.96663    -1.26284    1.61306 
  0.617865    -2.17288     1.82758    1.7016  
 -3.14557     -4.08182    -1.91

In [24]:
loss = resnet101(x,y)

10.4539585f0